### Skript zum Abgleich der Stichprobenvarianten

#### Bestimmung eines vergleichbaren Werts zur Evaluation unterschiedlicher Workflows

**Step 1:** Read in data sets -gt.txt and -out.txt

**Step 2:** Fill dict according to lines

**Step 3:** Check dict for the presence of ground truth lines

**Step 4:** Check the percent correctness (Sequence Matcher)

**Step 5:** Perform for all text files within certain folder & determine percentage value of correct output of ground truth content

In [1]:
# import OS module to use operating system dependent functionalities
import os
os.path

<module 'ntpath' from 'C:\\Users\\merle\\AppData\\Local\\Programs\\Python\\Python310\\lib\\ntpath.py'>

In [3]:
# create list of all files in ./Stichprobe_*
file_list = os.listdir("./Stichprobe_FILLOUT") # FILLOUT
# create separated list for -gt.txt and -out.txt files
file_list_gt=[]
file_list_out=[]
for file in file_list:
    if file.endswith("-gt.txt"):
        file_list_gt.append(file)
    else:
        file_list_out.append(file)

In [4]:
# check output files list
file_list_out

['tesseract-out.txt']

In [5]:
# check ground truth files list
file_list_gt

['tesseract-gt.txt']

In [6]:
# prepare dict with filename as key and lines as values (ground truth)
file_dict_gt= {}
for file_name in file_list_gt:
    file_dict_gt[file_name] = []

# for each file in list oft gt.txt files
for file_name in file_list_gt:    
    # open file
    file = open(f'./Stichprobe_FILLOUT/{file_name}',"r",encoding='utf-8') # FILLOUT
    # for each line in opened file
    for line in file:
        # remove \n
        line = line.strip("\n")
        if len(line) != 0:
            # add line to value of dict key of that file
            file_dict_gt[file_name].append(line)
    file.close()

print(file_dict_gt)


{'tesseract-gt.txt': ['Diese Textdatei enthält 5 Absätze, die Tesseract aus den Katalogen hätte herauslesen sollen.', 'Erster Name, zweiter Name und Name drei.', 'Wenn alles korrekt erkannt wird, wäre das super.', 'Best.-Nr. 000 Das hier könnte ein Titel sein. Super 8 Schwarz/Weiß was weiß ich (:D)', 'Genre']}


In [7]:
# prepare dict with filename as key and lines as values (output)
file_dict_out= {}
for file_name in file_list_out:
    file_dict_out[file_name] = []

# for each file in list oft text.txt files
for file_name in file_list_out:    
    # open file
    file = open(f'./Stichprobe_FILLOUT/{file_name}',"r",encoding='utf-8') # FILLOUT
    # for each line in file
    for line in file:
        # remove \n
        line = line.strip("\n")
        if len(line) != 0:
            # add line to value of dict key of that file
            file_dict_out[file_name].append(line)
    file.close()

print(file_dict_out)

{'tesseract-out.txt': ['Diese Textdatei enthält KEINE Absätze, die Tesseract aus den Katalogen hätte herauslesen sollen.', 'Erster Name, zweiter Name und Name drei.', 'Wenn alles korrekt erkannt wird, wäre das super.', 'Best.-Nr. 000 Das hier könnte ein Titel sein. Super 8 Schwarz/Weiß ', 'Genre']}


In [9]:
# create finished dict
result_dict = {}
for file_name in file_list_gt:
    result_dict[file_name.replace("-gt.txt", "")] = []
# check finished dict
result_dict

{'tesseract': []}

In [12]:
# SequenceMatcher: compare similarity of two strings by their ratio
from difflib import SequenceMatcher as SM

# iterate over keys in out.txt dict
for key in file_dict_gt:
    gt_value = file_dict_gt[key]
    # get beginning of file name and use it to extract value of out.txt dict
    out_value = file_dict_out[key.replace("gt", "out")]
    # for each pair of value compute distance measure 
    for string_gt in gt_value:
        print("*****************")
        print("This is gt value:")
        print(string_gt)     
        best_ratio = 0
        for string_out in out_value:
            print("--------------------------------")
            print("Try fitting with this out value:")
            print(string_out)
            similarity_ratio = SM(None, string_gt, string_out).ratio()
            print(f'The score is: {similarity_ratio}')
            if similarity_ratio >= best_ratio:
                best_ratio = similarity_ratio
                best_string_out = string_out
        result_dict[key.replace("-gt.txt", "")].append({
            'groundtruth_line':string_gt,
            'output_line': best_string_out,
            'similarity_ratio': best_ratio })

        

*****************
This is gt value:
Diese Textdatei enthält 5 Absätze, die Tesseract aus den Katalogen hätte herauslesen sollen.
--------------------------------
Try fitting with this out value:
Diese Textdatei enthält KEINE Absätze, die Tesseract aus den Katalogen hätte herauslesen sollen.
The score is: 0.9680851063829787
--------------------------------
Try fitting with this out value:
Erster Name, zweiter Name und Name drei.
The score is: 0.22727272727272727
--------------------------------
Try fitting with this out value:
Wenn alles korrekt erkannt wird, wäre das super.
The score is: 0.2
--------------------------------
Try fitting with this out value:
Best.-Nr. 000 Das hier könnte ein Titel sein. Super 8 Schwarz/Weiß 
The score is: 0.1761006289308176
--------------------------------
Try fitting with this out value:
Genre
The score is: 0.061855670103092786
*****************
This is gt value:
Erster Name, zweiter Name und Name drei.
--------------------------------
Try fitting with 

In [13]:
result_dict

{'tesseract': [{'groundtruth_line': 'Diese Textdatei enthält 5 Absätze, die Tesseract aus den Katalogen hätte herauslesen sollen.',
   'output_line': 'Diese Textdatei enthält KEINE Absätze, die Tesseract aus den Katalogen hätte herauslesen sollen.',
   'similarity_ratio': 0.9680851063829787},
  {'groundtruth_line': 'Erster Name, zweiter Name und Name drei.',
   'output_line': 'Erster Name, zweiter Name und Name drei.',
   'similarity_ratio': 1.0},
  {'groundtruth_line': 'Wenn alles korrekt erkannt wird, wäre das super.',
   'output_line': 'Wenn alles korrekt erkannt wird, wäre das super.',
   'similarity_ratio': 1.0},
  {'groundtruth_line': 'Best.-Nr. 000 Das hier könnte ein Titel sein. Super 8 Schwarz/Weiß was weiß ich (:D)',
   'output_line': 'Best.-Nr. 000 Das hier könnte ein Titel sein. Super 8 Schwarz/Weiß ',
   'similarity_ratio': 0.8874172185430463},
  {'groundtruth_line': 'Genre',
   'output_line': 'Genre',
   'similarity_ratio': 1.0}]}

In [14]:
average_page_dict= {}
for file_name in file_list_gt:
    average_page_dict[file_name.replace("-gt.txt", "")] = None

sum = 0
counter = 0
for key in average_page_dict:
    # entry is the list value of each page key
    for entry in result_dict[key]:
        #print(entry['similarity_ratio'])
        sum = sum + entry['similarity_ratio']
        counter = counter+1
    average_page_dict[key] = sum/counter

In [15]:
average_page_dict

{'tesseract': 0.9711004649852051}